In [1]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import scipy

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

import random

In [2]:
import layers as custom_layers

In [3]:
# We renamed cusolver64_11.dll to cusolver64_10.dll to solve the compatibility issue.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Load generated training data (by data loaders)

In [4]:
train_dataset_path = "./ToyTrainingData2/toy_affine{}.npz"
train_dataset_size = 50
img_shape = (64, 256, 256)
def data_generator(batch_size = 5):
    while True:
        img_pair_batch = np.zeros((batch_size, *img_shape, 2)).astype('float32')
        fixed_img_batch = np.zeros((batch_size, *img_shape, 1)).astype('float32')
        tgtAffineTrf_batch = np.zeros((batch_size, 12)).astype('float32')
        for i in range(batch_size):
            idx = random.randrange(train_dataset_size)
            inputObj = np.load(train_dataset_path.format(idx))
            img_pair = inputObj['img_pair']
            img_pair_batch[i] = img_pair[0]

            tgtAffineTrf = inputObj['trf']
            tgtAffineTrf_batch[i] = tgtAffineTrf
            fixed_img_batch[i] = np.expand_dims(img_pair[0, :, :, :, 0], -1)
        
#         yield (img_pair_batch, [fixed_img_batch, tgtAffineTrf_batch])
#         yield (img_pair_batch, tgtAffineTrf_batch)
        yield (img_pair_batch, fixed_img_batch)

## NN with labels as fixed images

In [5]:
leaky_relu_alpha = 0.3
def res_block(x, filters, kernel_size=(3, 3, 3), strides=(1, 1, 1)):
    x_skip = x
    f1, f2 = filters
    
    if f1 == f2:
        x = layers.Conv3D(filters=f1, kernel_size=kernel_size, strides=(1, 1, 1),
                          padding="same", kernel_regularizer=keras.regularizers.L2(l2=0.001))(x)
#         x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=leaky_relu_alpha)(x)
        
        x = layers.Conv3D(filters=f2, kernel_size=kernel_size, strides=(1, 1, 1),
                          padding="same", kernel_regularizer=keras.regularizers.L2(l2=0.001))(x)
#         x = layers.BatchNormalization()(x)
        
        x = layers.Add()([x, x_skip])
        x = layers.LeakyReLU(alpha=leaky_relu_alpha)(x)
        return x
    
    else:
        x = layers.Conv3D(filters=f1, kernel_size=kernel_size, strides=(1, 1, 1),
                          padding="same", kernel_regularizer=keras.regularizers.L2(l2=0.001))(x)
#         x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=leaky_relu_alpha)(x)
        
        x = layers.Conv3D(filters=f2, kernel_size=kernel_size, strides=strides,
                          padding="valid", kernel_regularizer=keras.regularizers.L2(l2=0.001))(x)
#         x = layers.BatchNormalization()(x)
        
        x_skip = layers.Conv3D(filters=f2, kernel_size=kernel_size, strides=strides,
                               padding="valid", kernel_regularizer=keras.regularizers.L2(l2=0.001))(x_skip)
        
        x = layers.Add()([x, x_skip])
        x = layers.LeakyReLU(alpha=leaky_relu_alpha)(x)
        return x
    

In [6]:
inputs = keras.Input(shape = (*img_shape, 2))
moving_input = tf.expand_dims(inputs[:, :, :, :, 1], axis = -1)
# fixed_input = tf.expand_dims(inputs[:, :, :, :, 1], axis = -1)

# down_depths = [4, 8, 16, 24, 32]
# up_depths = [2, 16, 32, 16, 3]

x = layers.Conv3D(filters=32, kernel_size=(7, 15, 15), strides=(2, 2, 2), activation="relu")(inputs) # relu or leaky_relu?
x = layers.MaxPool3D((1, 2, 2))(x)
print("0: {}".format(x.shape))

# First stage, no dimesnion changes
x = res_block(x, kernel_size=(3, 9, 9), filters=(32, 32))
x = res_block(x, kernel_size=(3, 9, 9), filters=(32, 32))
x = res_block(x, kernel_size=(3, 9, 9), filters=(32, 32))
print("1: {}".format(x.shape))

# Second stage, increase channel depth, halve the height and width
x = res_block(x, kernel_size=(3, 9, 9), filters=(32, 64), strides=(1, 2, 2))
x = res_block(x, filters=(64, 64))
x = res_block(x, filters=(64, 64))
x = res_block(x, filters=(64, 64))
print("2: {}".format(x.shape))

# Third stage
x = res_block(x, filters=(64, 128), strides=(2, 2, 2))
x = res_block(x, filters=(128, 128))
x = res_block(x, filters=(128, 128))
x = res_block(x, filters=(128, 128))
print("3: {}".format(x.shape))

# Fourth stage
x = res_block(x, filters=(128, 256), strides=(2, 2, 2))
x = res_block(x, filters=(256, 256))
x = res_block(x, filters=(256, 256))
x = res_block(x, filters=(256, 256))
print("4: {}".format(x.shape))

# f_conv_0 = layers.Conv3D(filters=down_depths[0], kernel_size=(6, 32, 32), strides=(1, 2, 2), activation="relu")(fixed_input)
# f_conv_0 = layers.BatchNormalization()(f_conv_0)
# f_conv_1 = layers.Conv3D(filters=down_depths[1], kernel_size=(6, 16, 16), strides=(2, 2, 2), activation="relu")(f_conv_0)
# f_conv_1 = layers.BatchNormalization()(f_conv_1)
# # f_filtered_conv_1 = layers.MaxPool3D((2, 2, 2))(f_conv_1)
# f_conv_2 = layers.Conv3D(filters=down_depths[2], kernel_size=(6, 8, 8), activation="relu")(f_conv_1)
# f_conv_2 = layers.BatchNormalization()(f_conv_2)
# # f_filtered_conv_2 = layers.MaxPool3D((1, 2, 2))(f_conv_2)
# f_conv_3 = layers.Conv3D(filters=down_depths[3], kernel_size=(4, 8, 8), strides=(1, 2, 2), activation="relu")(f_conv_2)
# f_conv_3 = layers.BatchNormalization()(f_conv_3)
# f_conv_4 = layers.Conv3D(filters=down_depths[4], kernel_size=(4, 4, 4), activation="relu")(f_conv_3)
# f_conv_4 = layers.BatchNormalization()(f_conv_4)

# concat_feats = tf.concat([conv_4, f_conv_4], axis = -1)
# print(concat_feats.shape)

x = layers.AveragePooling3D((2, 2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512)(x)
# x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Dense(64)(x)
# x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
affine_pred = layers.Dense(12, activation="linear", name="affine_pred")(x)

# convTransposed_3 = layers.Conv3DTranspose(filters=up_depths[0], kernel_size=(6, 10, 10), activation="relu")(conv_4)
# print("convTransposed_3: {}".format(convTransposed_3.shape))
# filtered_convTransposed_2 = layers.Conv3DTranspose(filters=up_depths[1], kernel_size=(6, 20, 20), activation="relu")(convTransposed_3)
# print("filtered_convTransposed_2: {}".format(filtered_convTransposed_2.shape))
# convTransposed_2 = layers.UpSampling3D((1, 2, 2))(filtered_convTransposed_2)
# print("convTransposed_2: {}".format(convTransposed_2.shape))
# filtered_convTransposed_1 = layers.Conv3DTranspose(filters=up_depths[2], kernel_size=(3, 15, 15), activation="relu")(convTransposed_2)
# print("filtered_convTransposed_1: {}".format(filtered_convTransposed_1.shape))
# convTransposed_1 = layers.UpSampling3D((2, 2, 2))(filtered_convTransposed_1)
# print("convTransposed_1: {}".format(convTransposed_1.shape))
# convTransposed_0 = layers.Conv3DTranspose(filters=up_depths[3], kernel_size=(3, 16, 16), activation="relu")(convTransposed_1)
# print("convTransposed_0: {}".format(convTransposed_0.shape))
# deformation_field_pred = layers.Conv3DTranspose(filters=up_depths[4], kernel_size=(3, 10, 10), activation="relu")(convTransposed_0)
# print("deformation_field_pred: {}".format(deformation_field_pred.shape))

affine_warped = custom_layers.SpatialTransformer(interp_method='linear', 
                                                 add_identity=False, name="warped_image", 
                                                 shift_center=True)([moving_input, affine_pred])
# deformable_warped = custom_layers.SpatialTransformer(interp_method='linear', add_identity=False, name="warped_image", shift_center=False)([affine_warped, deformation_field_pred])
print(affine_warped.shape)

0: (None, 29, 60, 60, 32)
1: (None, 29, 60, 60, 32)
2: (None, 27, 26, 26, 64)
3: (None, 13, 12, 12, 128)
4: (None, 6, 5, 5, 256)
Instructions for updating:
Use fn_output_signature instead
(None, 64, 256, 256, 1)


In [ ]:
print(moving_input.shape)

In [7]:
# model = keras.Model(inputs=inputs, outputs=[affine_warped, affine_pred], name="combined_model")
# model = keras.Model(inputs=inputs, outputs=affine_pred, name="combined_model")
model = keras.Model(inputs=inputs, outputs=affine_warped, name="combined_model")
model.summary()

Model: "combined_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 256, 256 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 29, 121, 121, 100832      input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling3d (MaxPooling3D)    (None, 29, 60, 60, 3 0           conv3d[0][0]                     
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 29, 60, 60, 3 248864      max_pooling3d[0][0]              
_____________________________________________________________________________________

In [ ]:
dataGen = data_generator(batch_size = 1)
moving_test, label_test = next(dataGen)
print(moving_test.shape)
(warped_test, affine_pred_test) = model(moving_test)
print(label_test[1])
print(label_test[1].dtype)
print(affine_pred_test)
mse = tf.keras.losses.MeanSquaredError()
print(mse(label_test[0], warped_test))
print(mse(label_test[1], affine_pred_test))

In [ ]:
# keras.utils.plot_model(model, "test.png", show_shapes=True)

In [8]:
lr = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
# loss_object = tf.keras.losses.MeanSquaredError()

save_callback = ModelCheckpoint(filepath='./checkpoints/{epoch:02d}.h5',
                                save_freq=5)

In [ ]:
"""
y = (deformed_img, affine_trf)
"""
def affine_loss(y_actual, y_pred):
    y_actual = y_actual[0]
    y_pred = y_pred[0]
    tgtDiag = [y_actual[0]] + [y_actual[5]] + [y_actual[10]]
    predDiag = [y_pred[0]] + [y_pred[5]] + [y_pred[10]]
    tgtCorner = y_actual[1:3] + [y_actual[4]] + [y_actual[6]] + y_actual[8:10]
    predCorner = y_pred[1:3] + [y_pred[4]] + [y_pred[6]] + y_pred[8:10]
    tgtTranslation = [y_actual[3]] + [y_actual[7]] + [y_actual[11]]
    predTranslation = [y_pred[3]] + [y_pred[7]] + [y_pred[11]]
    mse = tf.keras.losses.MeanSquaredError()
    diag_loss = mse(tgtDiag, predDiag)
    corner_loss = mse(tgtCorner, predCorner)
    translation_loss = mse(tgtTranslation, predTranslation)
    return corner_loss * 10 + diag_loss * 10 + translation_loss

In [9]:
model.compile(optimizer=optimizer,
#               loss={"warped_image":"mean_squared_error", "affine_pred":"mean_squared_error"},
#               loss={"affine_pred":"mean_squared_error"},
              loss={"warped_image":"mean_squared_error"},
#               loss_weights={"warped_image":1, "affine_pred":1},
              run_eagerly=True)

In [10]:
batch_size = 2
dataGen = data_generator(batch_size)

In [11]:
history = model.fit(dataGen, epochs=100, steps_per_epoch=train_dataset_size/batch_size, callbacks=[save_callback])

Epoch 1/100
25/25 [==============================] - 49s 1s/step - loss: 3.9565
Epoch 2/100
25/25 [==============================] - 28s 1s/step - loss: 3.0870
Epoch 3/100
25/25 [==============================] - 27s 1s/step - loss: 3.5709
Epoch 4/100
25/25 [==============================] - 26s 1s/step - loss: 3.6252
Epoch 5/100
25/25 [==============================] - 26s 1s/step - loss: nan
Epoch 6/100
 8/25 [========>.....................] - ETA: 15s - loss: nan

KeyboardInterrupt: 

## Test model (output == warpedImg)

In [ ]:
testModel = keras.Model(inputs=inputs, outputs=[affine_warped, affine_pred])

In [ ]:
# testModel.load_weights('./checkpoints/epoch_{}'.format(epochs-1))
testModel.load_weights('./checkpoints/19.h5')

In [ ]:
dataGen_test = data_generator(batch_size = 5)
moving_test, label_test = next(dataGen_test)
(warped_test, affine_pred_test) = testModel(moving_test)
print(label_test[1])
print(affine_pred_test)
mse = tf.keras.losses.MeanSquaredError()
print(mse(label_test[1], affine_pred_test))

In [ ]:
sliceToCheck = 0
fig, axs = plt.subplots(5, 3, figsize=(15, 35))
for i in range(5):
    axs[i, 0].imshow(fixedImg[sliceToCheck + i * 5,:,:,0])
    axs[i, 0].set_title("Fixed Image")
    axs[i, 1].imshow(moving_test[0,sliceToCheck + i * 5,:,:,0])
    axs[i, 1].set_title("Moving Image")
    axs[i, 2].imshow(warped_test[0,sliceToCheck + i * 5,:,:,0])
    axs[i, 2].set_title("Warped Image")
plt.show()